In [ ]:
!pip install ftfy regex tqdm
import os
import torch
from PIL import Image
import torchvision.transforms as T
from urllib import request

In [21]:
vocab_url = "https://github.com/JoyBingzhi/clip-image-text-matching/raw/main/bpe_simple_vocab_16e6.txt.gz"
dog_url = "https://github.com/JoyBingzhi/clip-image-text-matching/raw/main/test_images/dog.jpg"
plane_url = "https://github.com/JoyBingzhi/clip-image-text-matching/raw/main/test_images/airplane.jpg"
clip_url = "https://github.com/JoyBingzhi/clip-image-text-matching/raw/main/custom_clip.py"
model_url = "https://github.com/JoyBingzhi/clip-image-text-matching/raw/main/custom_model.py"
tokenizer_url = "https://github.com/JoyBingzhi/clip-image-text-matching/raw/main/custom_tokenizer.py"


In [25]:
request.urlretrieve(vocab_url, "bpe_simple_vocab_16e6.txt.gz")
request.urlretrieve(clip_url, "custom_clip.py")
request.urlretrieve(model_url, "custom_model.py")
request.urlretrieve(tokenizer_url, "custom_tokenizer.py")
from custom_clip import load
from custom_tokenizer import SimpleTokenizer

In [29]:

# 4. 加载模型
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _ = load(name="ViT-B/32", device=device, jit=False)

# 加载预训练权重
state_dict = torch.load("clip-imp-pretrained_128_6_after_4.pt", map_location=device)
model.load_state_dict(state_dict)
model.eval()

# 5. 图像预处理
preprocess = T.Compose([
    T.Resize(224, interpolation=Image.BICUBIC),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711]),
])

images = []
for path in ["dog.jpg", "airplane.jpg"]:
    img = Image.open(path).convert("RGB")
    images.append(preprocess(img).unsqueeze(0))
images = torch.cat(images).to(device)

# 6. 文本处理
texts = ["This is a dog.", "This is an airplane."]
tokenizer = SimpleTokenizer()
tokenized = [tokenizer.encode(t) for t in texts]
# max_len = max(len(t) for t in tokenized)
max_len = 77
text_tensor = torch.zeros(len(texts), max_len, dtype=torch.long).to(device)
for i, tokens in enumerate(tokenized):
    text_tensor[i, :len(tokens)] = torch.tensor(tokens)

# 7. 提取特征
with torch.no_grad():
    image_features = model.encode_image(images)
    text_features = model.encode_text(text_tensor)

# 8. 计算相似度
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)

similarity = (100.0 * image_features @ text_features.T)

# 9. 输出结果
for i, path in enumerate(["dog.jpg", "airplane.jpg"]):
    print(f"Image: {path}")
    for j, text in enumerate(texts):
        print(f"  '{text}': {similarity[i][j].item():.2f}")

Image: dog.jpg
  'This is a dog.': 28.22
  'This is an airplane.': 25.43
Image: airplane.jpg
  'This is a dog.': 20.38
  'This is an airplane.': 24.84
